# Analyzing a Text Data Set
This notebook creates a text data set using a Twitter API and analyzes the data using some parsing.
My goal for this assignment was to utilize methods that I am knowledgeable of on a data set that interests me - aka National Parks.

#### A New Angle Soundcloud data note:
Originally, I planned to complete this assignment using some A New Angle text data, but I wasn't able to compile the dataset and do the analysis in the time remaining with my current skillset. When I have more time and improve my coding skills, I hope to continue that project. Sorry John, I know that you really wanted me to stick with the pod data. Trust me, I did to.

## Step 1: Creates a text data set using Twitter API

In [ ]:
#Imports for creating text data set
from datetime import datetime
import json
from pprint import pprint

from twitter_functions import * # these are the functions John wrote for ADA.

In [ ]:
#My Twitter credentials
auth =  { "consumer_key": "OnllVAvDyo1P59VygbJaRRctU", 
          "consumer_secret": "2d0DspkrPRS9aCwemPx3U13gd7IpZr0ASSqdCy1ZaP3ALlqvx3",
          "access_key": "1169449721665974273-GP9tiIGcfZ7weAoKEub4Kxdk4D5hdm",
          "access_secret": "PHbUPHSdxn1og9OrCcIlE5bacFRLTI1BPooqKxvwvBOQ3"}

api = initialize_twitter(auth)

The following cells pull all the Twitter followers of Glacier National Park (263.5K) and Grand Canyon National Park (133.2K). 

Then it pulls the following Twitter data of those users:
* screen_name
* name
* id
* location
* followers_count
* friends_count
* description

and then writes this to a text file for analysis.

In [ ]:
#twitter_user = ['GlacierNPS'] # group1 - Glacier National Park (263.5K Followers) - 0.88hrs est. time
twitter_user = ['GrandCanyonNPS'] # group2 - Grand Canyon NPS (133.2K Followers) - 0.44hrs est. time

ofile_name = (datetime.today().strftime("%Y%m%d") + "_" + 
             starting_user[0] + "_" +
             "followers.txt")

In [ ]:
# Looks up the full information on my starting user(s) (GNP or GCNP)
starting_user_id = []

# All records will be a dictionary with the twitter ID as the key and 
# a UserRecord as the value. This is a named tuple John created in twitter_functions.py. 
all_records = lookup_users_from_handles(api, starting_user)

# Creates list of IDs that we are getting followers from 
for id in all_records : #access the keys, which are ids.
    starting_user_id.append(id)

In [ ]:
all_records

This was super helpful as I originally planned to pull the U.S. Soccer WNT (@USWNT) 2.5M Followers and the U.S. Soccer MNT (@USMNT) 2.2M. A complete pull WITHOUT run limits would have take 7-8 hours each. I simply didn't have the time and scaled back.

In [ ]:
# Estimates time it will take to pull these followers
total_followers = 0
for id, rec in all_records.items() :
    total_followers += rec.followers_count
    
print("Ooh, {fol:,d} followers. A complete run with no limits run is ".format(fol=total_followers) + 
      "going to take {min:.2f} minutes ({hour:.2f} hours)".format(min=total_followers/5000,
                                                                  hour=total_followers/(60*5000)))

And now we go, go, go!

In [ ]:
# This pulls all the followers of our starting_user (GNP or GCNP)

followers_of_starting = gather_followers(api,
                                         starting_user_id,
                                         follower_limit=None)  

# followers_of_starting will be a dictionary with the key being the id(s) in starting_user_id
# and the value is a list of all the followers.

In [ ]:
# This "hydrates" these user records with screen_name, name, id, location, followers_count, friends_count, description
for start_id, list_of_followers in followers_of_starting.items() :

    ids_to_hydrate = {id for id in list_of_followers if id not in all_records} 
    
    these_records = lookup_users_from_ids(api,ids=ids_to_hydrate)

    for id, rec in these_records.items() :
        all_records[id] = rec

In [ ]:
# This writes out all the records to a text file.
with open(ofile_name,'w') as ofile :
    write_user_rec_headers(ofile)
    for id, rec in all_records.items() :
        write_user_rec(ofile, rec)

### END Step 1 - Create text data set using Twitter API

## Step 2: Analysis of a text data set

In [ ]:
#Imports for analyzing text data set
import nltk
from nltk.probability import FreqDist

### Parsing - twitter descriptions

In [ ]:
#reads in text files (descriptions) created in step 1

file_name = "20191205_GlacierNPS_followers.txt"
#file_name = "20191205_GrandCanyonNPS_followers.txt"

descs = []
with open(file_name,'r') as ifile :
    next(ifile)
    
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split("\t")
        
        # spot 6 has the description
        if len(line) >= 7 : # sometimes we don't have descriptions
            descs.extend(line[6].split())
        
        # for now we'll just add on to a big list

In [ ]:
with open(file_name,'r') as ifile :
    print(ifile.readline()) #prints "headers"
    print(ifile.readline()) #prints corresponding info

Cast words to lowercase, removes stopwords, and includes only alphanumeric words

In [ ]:
stopwords = set(nltk.corpus.stopwords.words("english"))
stopwords_sp = set(nltk.corpus.stopwords.words("spanish"))

def clean_list(text) :
    ''' takes a list of text and returns a new list with 
        * words cast to lowercase
        * stopwords removed
        * only alphanumeric words
    '''
    text_clean = [w.lower() for w in text if w.isalpha()]
    text_clean = [w for w in text_clean if w not in stopwords]
    text_clean = [w for w in text_clean if w not in stopwords_sp]
    return(text_clean)

In [ ]:
#clean list
descs_clean_gnp = clean_list(descs) #Glacier National Park
#descs_clean_gcnp = clean_list(descs) #Grand Canyon National Park

In [ ]:
print(descs_clean_gnp.count("montana")) #number of times word was used
print(100 * descs_clean_gnp.count("montana")/len(descs_clean_gnp)) #percent

#print(descs_clean_gcnp.count("life")) #number of times word was used
#print(100 * descs_clean_gcnp.count("life")/len(descs_clean_gcnp)) #percent

In [ ]:
#Frequency Distribution
fd_gnp = FreqDist(descs_clean_gnp)
fd_gnp.most_common(50)

#fd_gcnp = FreqDist(descs_clean_gcnp)
#fd_gcnp.most_common(50)

### Glacier National Park
Number of times word was used:
* montana = 1282 (0.1951706677668487%)
* mt = 294 (0.04475832786540836%)
* arizona = 121 (0.018420944461613645%)
* az = 70 (0.010656744729859132%)
* love = 6547 (0.9967101106626821%)
* life = 3494 (0.5319238012303973%)

Frequency distribution: (top 50)

[('love', 6547),
 ('life', 3494),
 ('news', 3411),
 ('music', 2678),
 ('politics', 2592),
 ('science', 2409),
 ('world', 2385),
 ('lover', 2324),
 ('sports', 2311),
 ('like', 2199),
 ('travel', 2124),
 ('national', 1937),
 ('government', 1933),
 ('one', 1831),
 ('us', 1817),
 ('new', 1717),
 ('living', 1653),
 ('good', 1652),
 ('things', 1651),
 ('proud', 1617),
 ('mom', 1550),
 ('live', 1538),
 ('people', 1521),
 ('business', 1500),
 ('make', 1477),
 ('great', 1458),
 ('fan', 1420),
 ('time', 1416),
 ('nature', 1392),
 ('student', 1365),
 ('follow', 1356),
 ('family', 1318),
 ('best', 1313),
 ('outdoor', 1295),
 ('montana', 1282),
 ('go', 1252),
 ('university', 1227),
 ('state', 1188),
 ('art', 1170),
 ('get', 1161),
 ('retired', 1156),
 ('twitter', 1153),
 ('tweets', 1147),
 ('technology', 1131),
 ('dog', 1127),
 ('social', 1124),
 ('country', 1114),
 ('always', 1100),
 ('former', 1068),
 ('college', 1058)]

### Grand Canyon National Park
Number of times word was used:
* arizona = 545 (0.11823690336465345%)
* az = 274 (0.0594438743521377%)
* montana = 109 (0.023647380672930693%)
* mt = 36 (0.007810144075463348%)
* love = 5031 (1.0914676345460028%)
* life = 2711 (0.5881472385716983%)

Frequency distribution: (top 50)

[('love', 5031),
 ('life', 2711),
 ('lover', 2013),
 ('travel', 1980),
 ('world', 1753),
 ('music', 1708),
 ('national', 1691),
 ('like', 1638),
 ('news', 1575),
 ('new', 1421),
 ('one', 1415),
 ('us', 1328),
 ('sports', 1324),
 ('living', 1307),
 ('things', 1305),
 ('fan', 1287),
 ('science', 1281),
 ('proud', 1270),
 ('mom', 1269),
 ('good', 1183),
 ('nature', 1177),
 ('live', 1173),
 ('make', 1052),
 ('time', 1049),
 ('family', 1040),
 ('great', 1032),
 ('tweets', 1030),
 ('people', 1030),
 ('follow', 1029),
 ('outdoor', 1022),
 ('go', 972),
 ('best', 961),
 ('student', 938),
 ('photographer', 928),
 ('twitter', 913),
 ('opinions', 912),
 ('park', 900),
 ('state', 892),
 ('always', 885),
 ('university', 882),
 ('loves', 881),
 ('former', 881),
 ('social', 878),
 ('dog', 876),
 ('art', 870),
 ('get', 851),
 ('views', 830),
 ('public', 824),
 ('teacher', 814),
 ('business', 808)]

### Parsing - twitter location

In [ ]:
#reads in text files (descriptions) created in step 1

file_name = "20191205_GlacierNPS_followers.txt"
#file_name = "20191205_GrandCanyonNPS_followers.txt"

location = []
with open(file_name,'r') as ifile :
    next(ifile)
    
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split("\t")
        
        # spot 3 has the location
        if len(line) >= 7 : # sometimes we don't have descriptions
            location.extend(line[3].split())
        
        # for now we'll just add on to a big list

In [ ]:
#location

In [ ]:
with open(file_name,'r') as ifile :
    print(ifile.readline()) #prints "headers"
    print(ifile.readline()) #prints corresponding info

In [ ]:
def clean_list(text) :
    ''' takes a list of text and returns a new list with 
        * words cast to lowercase
        * only alphanumeric words
    '''
    text_clean = [w.lower() for w in text if w.isalpha()]
    return(text_clean)

In [ ]:
#clean list
loc_clean_gnp = clean_list(location) #Glacier National Park
#loc_clean_gcnp = clean_list(location) #Grand Canyon National Park

In [ ]:
print(loc_clean_gnp.count("grand canyon")) #number of times word was used
print(100 * loc_clean_gnp.count("grand canyon")/len(loc_clean_gnp)) #percent

#print(loc_clean_gcnp.count("canyon")) #number of times word was used
#print(100 * loc_clean_gcnp.count("canyon")/len(loc_clean_gcnp)) #percent

In [ ]:
#Frequency Distribution
fd_gnp = FreqDist(loc_clean_gnp)
fd_gnp.most_common(50)

#fd_gcnp = FreqDist(loc_clean_gcnp)
#fd_gcnp.most_common(50)

### Glacier National Park
Number of times location was:
* montana = 1559 (1.1696213547801426%)
* mt = 3698 (2.774380865924931%)
* missoula = 103 (0.07727453466475606%)
* arizona = 224 (0.1680533569408287%)
* az = 681 (0.5109122146281444%)

Frequency distribution: (top 50)

[('usa', 11209),
 ('ca', 4013),
 ('mt', 3698),
 ('united', 3185),
 ('states', 2993),
 ('new', 2309),
 ('tx', 1925),
 ('san', 1669),
 ('pakistan', 1655),
 ('wa', 1636),
 ('ny', 1580),
 ('montana', 1559),
 ('the', 1491),
 ('fl', 1362),
 ('of', 1311),
 ('co', 1186),
 ('los', 1144),
 ('il', 1119),
 ('in', 1077),
 ('dc', 1012),
 ('pa', 1004),
 ('nc', 908),
 ('oh', 831),
 ('mn', 819),
 ('north', 792),
 ('ga', 790),
 ('california', 769),
 ('or', 755),
 ('tn', 725),
 ('ma', 689),
 ('az', 681),
 ('va', 665),
 ('republic', 654),
 ('south', 650),
 ('mi', 617),
 ('texas', 614),
 ('mo', 586),
 ('colorado', 564),
 ('wi', 539),
 ('west', 518),
 ('bangladesh', 515),
 ('earth', 478),
 ('canada', 475),
 ('florida', 469),
 ('la', 465),
 ('ut', 462),
 ('iraq', 460),
 ('lake', 459),
 ('york', 457),
 ('s', 448)]

### Grand Canyon National Park
Number of times location was:
* montana = 125 (0.13259645065820877%)
* mt = 282 (0.299137592684919%)
* missoula = 6 (0.006364629631594021%)
* arizona = 834 (0.884683518791569%)
* az = 2131 (2.2605042908211432%)

Frequency distribution: (top 50)

[('usa', 7645),
 ('ca', 3809),
 ('az', 2131),
 ('united', 2097),
 ('states', 1911),
 ('new', 1817),
 ('san', 1565),
 ('tx', 1562),
 ('the', 1389),
 ('los', 1204),
 ('ny', 1170),
 ('fl', 1011),
 ('co', 965),
 ('california', 918),
 ('wa', 916),
 ('in', 846),
 ('arizona', 834),
 ('il', 817),
 ('pa', 789),
 ('dc', 745),
 ('of', 739),
 ('nc', 629),
 ('oh', 612),
 ('ga', 606),
 ('texas', 601),
 ('north', 595),
 ('las', 587),
 ('south', 578),
 ('tn', 575),
 ('ma', 549),
 ('va', 547),
 ('england', 522),
 ('ut', 515),
 ('nv', 511),
 ('colorado', 498),
 ('or', 490),
 ('mo', 455),
 ('york', 440),
 ('lake', 435),
 ('mi', 430),
 ('earth', 429),
 ('mn', 428),
 ('florida', 398),
 ('canada', 384),
 ('city', 377),
 ('angeles', 373),
 ('west', 367),
 ('la', 361),
 ('ohio', 359),
 ('utah', 346)]

### END - Analysis of a text data set.